In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
import os
from sklearn.model_selection import train_test_split
import skimage
from keras.applications import DenseNet121
from keras.applications.densenet import conv_block, dense_block

patch_dir = "D:\ISEN\M1\Projet M1\DLBCL-Morph\Patches\HE"
extracted_features_save_adr = "./extracted_features.pickle"
network_weights_address = "./weights/KimiaNetKerasWeights.h5"
network_input_patch_width = 224
batch_size = 32
img_format = 'png'

In [8]:
def create_train_dataset(patch_dir):
    patient_df = pd.read_csv('D:\ISEN\M1\Projet M1\KimiaNet\clinical_data_with_no_missing_values.csv')
    y_data = patient_df[['patient_id', 'Follow-up Status']].copy()
    train_dataset = []
    for dirs in os.listdir(patch_dir):
        patient_id = dirs
        for files in os.listdir(patch_dir + "\\" + dirs):
            if files.endswith(".png"):
                image_data = skimage.io.imread(fname = patch_dir + "\\" + dirs + "\\" + files, as_gray = True)
                train_dataset.append([image_data, patient_id])

    x_train = []
    y_train = []
    for i in range(len(train_dataset)):
        x_train.append(train_dataset[i][0])
        y_data.loc[:, 'patient_id'] = y_data['patient_id'].astype(str)
        y_train.append(y_data[y_data['patient_id'] == train_dataset[i][1]]['Follow-up Status'].values[0])
    return x_train, y_train

In [9]:
# Create the train dataset
X, y = create_train_dataset(patch_dir)
x_train, x_test, y_train, y_test = train_test_split(X,y ,random_state=104, test_size=0.2, shuffle=True)

In [10]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [11]:
# Normalize the images
x_train_norm = x_train.astype('float32') / 255.
x_test_norm = x_test.astype('float32') / 255.

In [14]:
x_train_norm = x_train_norm[..., tf.newaxis]
x_test_norm = x_test_norm[..., tf.newaxis]

(24005, 224, 224, 1, 1)

In [21]:
x_train_norm = x_train_norm.reshape((24005,224,224,1))
x_test_norm = x_test_norm.reshape((6002,224,224,1))

In [22]:
print(x_train_norm.shape)
print(x_test_norm.shape)

(24005, 224, 224, 1)
(6002, 224, 224, 1)


In [1]:
# Print a sample image in grayscale
plt.imshow(x_train_norm[0, ..., 0], cmap='gray')

NameError: name 'plt' is not defined

In [1]:
# # Add noise to the images
# x_train_noisy = x_train_norm.copy()
# x_test_noisy = x_test_norm.copy()
# for i in range(len(x_train_norm)):
#     # Select a random noise factor between 0 and 0.2
#     noise_factor = np.random.uniform(0, 0.2)
#     # Add noise to the image
#     x_train_noisy[i] = x_train_norm[i] + noise_factor * tf.random.normal(shape=x_train_norm.shape)
# for i in range(len(x_test_norm)):
#     # Select a random noise factor between 0 and 0.2
#     noise_factor = np.random.uniform(0, 0.2)
#     # Add noise to the image
#     x_test_noisy[i] = x_test_norm[i] + noise_factor * tf.random.normal(shape=x_test_norm.shape)

NameError: name 'x_train_norm' is not defined

In [27]:
noise_factor = 0.2
x_train_noisy = x_train_norm + noise_factor * tf.random.normal(shape=x_train_norm.shape)
x_test_noisy = x_test_norm + noise_factor * tf.random.normal(shape=x_test_norm.shape)

x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

ResourceExhaustedError: {{function_node __wrapped__RandomStandardNormal_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[24005,224,224,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomStandardNormal]

In [4]:
dnx = DenseNet121(include_top=False, weights=network_weights_address,
                      input_shape=(network_input_patch_width, network_input_patch_width, 3), pooling='avg')

In [5]:
dnx.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                        

In [6]:
reduced_dnx = Model(inputs=dnx.input, outputs=dnx.get_layer('pool4_pool').output)
reduced_dnx.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [3]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Lambda
from tensorflow.keras.backend import bias_add, constant

def preprocessing_fn(input_batch, network_input_patch_width):
    org_input_size = tf.shape(input_batch)[1]

    # standardization
    scaled_input_batch = tf.cast(input_batch, 'float') / 255.

    # resizing the patches if necessary
    resized_input_batch = tf.cond(tf.equal(org_input_size, network_input_patch_width),
                                  lambda: scaled_input_batch,
                                  lambda: tf.image.resize(scaled_input_batch,
                                                          (network_input_patch_width, network_input_patch_width)))

    # normalization, this is equal to tf.keras.applications.densenet.preprocess_input()---------------
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    data_format = "channels_last"
    mean_tensor = constant(-np.array(mean))
    standardized_input_batch = bias_add(resized_input_batch, mean_tensor, data_format)
    standardized_input_batch /= std



In [4]:
from keras import Input
from keras.layers import Flatten
from tensorflow.keras import models
from tensorflow.keras.applications.densenet import preprocess_input

#kn_feature_extractor = Model(inputs=dnx.input, outputs=dnx.layers[-5].output)
i = Input(shape=(network_input_patch_width, network_input_patch_width, 3), dtype=tf.uint8)
x = tf.cast(i, tf.float32)
x = preprocess_input(x)
x = reduced_dnx(x)

# Define the autoencoder model
encoder = Model(i, x)

encoder.summary()

NameError: name 'reduced_dnx' is not defined

In [21]:
from keras import backend
from keras.layers import UpSampling2D

def transition_block_transpose(x, dilation, name):
    bn_axis = 3
    x = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name=name + "_bn"
    )(x)
    x = layers.Activation("relu", name=name + "_relu")(x)
    x = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    x = layers.Conv2DTranspose(
        int(backend.int_shape(x)[bn_axis] * dilation),
        1,
        use_bias=False,
        name=name + "_conv",
    )(x)
    return x

In [22]:
def dense_block_transpose(x, blocks, name):
    for i in range(blocks):
        x = conv_block(x, 32, name=name + "_block" + str(i + 1))
    return x

In [23]:
def conv_block_transpose(x, growth_rate, name):
    bn_axis = 3 if backend.image_data_format() == "channels_last" else 1
    x1 = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name=name + "_0_bn"
    )(x)
    x1 = layers.Activation("relu", name=name + "_0_relu")(x1)
    x1 = layers.Conv2D(4 * growth_rate, 1, use_bias=False, name=name + "_1_conv")(x1)
    x1 = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name=name + "_1_bn"
    )(x1)
    x1 = layers.Activation("relu", name=name + "_1_relu")(x1)
    x1 = layers.Conv2D(growth_rate, 3, padding="same", use_bias=False, name=name + "_2_conv")(x1)
    x = layers.Concatenate(axis=bn_axis, name=name + "_concat")([x, x1])
    return x

In [29]:
from keras import backend

from keras.layers import Conv2DTranspose, Conv2D
decoder_blocks =  [24,12,6]
i = encoder.layers[-1].output
x = transition_block_transpose(i,0.5, "transition_block1")
x = dense_block(x, decoder_blocks[0], "dense_block1")
x = transition_block_transpose(x, 0.5, "transition_block2")
x = dense_block(x, decoder_blocks[1], "dense_block2")
x = transition_block_transpose(x, 0.5, "transition_block3")
x = dense_block(x, decoder_blocks[2], "dense_block3")
x = layers.Conv2D(64, 3, padding="same", use_bias=False)(x)


# decoder = models.Sequential()
# for layers in reversed(encoder.layers[-1].layers):
#     if isinstance(layers, Conv2D):
#         layer = Conv2DTranspose(filters=layers.filters, kernel_size=layers.kernel_size, strides=layers.strides,
#                                 padding=layers.padding, activation=layers.activation)
#     else:
#         layer = layers
#     decoder.add(layer)
decoder = Model(i, x)
decoder.summary()


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 7, 7, 512)]  0           []                               
                                                                                                  
 transition_block1_bn (BatchNor  (None, 7, 7, 512)   2048        ['input_9[0][0]']                
 malization)                                                                                      
                                                                                                  
 transition_block1_relu (Activa  (None, 7, 7, 512)   0           ['transition_block1_bn[1][0]']   
 tion)                                                                                            
                                                                                            

In [27]:
autoencoder = models.Sequential([encoder, decoder])
autoencoder.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        (None, 7, 7, 512)         4847168   
                                                                 
 model_6 (Functional)        (None, 56, 56, 640)       6121088   
                                                                 
Total params: 10,968,256
Trainable params: 10,846,400
Non-trainable params: 121,856
_________________________________________________________________


In [ ]:
input = encoder.input
output = [encoder.layers[21].output, ..., encoder.layers[-1].output]
encoder  = Model(input, output)

In [5]:
intermediate_ft = []
i = Input(shape=(32, 32, 3), dtype=tf.float32)
x = layers.Conv2D(8, 2, use_bias=False)(i)
intermediate_ft.append(x)
x = layers.Conv2D(16, 2, use_bias=False)(x)
intermediate_ft.append(x)
x = layers.Conv2D(32, 2, use_bias=False)(x)
intermediate_ft.append(x)
encoder = Model(i, intermediate_ft)
encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 31, 31, 8)         96        
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 16)        512       
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 29, 32)        2048      
                                                                 
Total params: 2,656
Trainable params: 2,656
Non-trainable params: 0
_________________________________________________________________


In [6]:
x = layers.Conv2DTranspose(32, 2, use_bias=False)(intermediate_ft[-1])
x = layers.Concatenate(axis=-1)([x, intermediate_ft[-2]])
x = layers.Conv2DTranspose(16, 2, use_bias=False)(x)
x = layers.Concatenate(axis=-1)([x, intermediate_ft[-3]])
x = layers.Conv2DTranspose(8, 2, use_bias=False)(x)
decoder = Model(intermediate_ft, x)
decoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 29, 29, 32)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 30, 30, 32)  4096        ['input_5[0][0]']                
 ose)                                                                                             
                                                                                                  
 input_4 (InputLayer)           [(None, 30, 30, 16)  0           []                               
                                ]                                                           